# Lesson 6: Table Joins

**Duration:** 20 minutes  
**Prerequisites:** Complete Lessons 1-5  
**Learning Mode:** Read explanations, then run each SQL query

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
- Understand why we need JOINs
- Write INNER JOIN queries
- Write LEFT JOIN queries
- Combine data from multiple tables
- Handle NULL values in joins
- Use joins with WHERE, ORDER BY, and aggregate functions


## 📚 What are JOINs?

**JOINs** combine rows from two or more tables based on a related column. They allow you to retrieve data spread across multiple tables in a single query.

### The Problem:

Currently, our data is split across tables:
- `characters` table has `name` and `homeworld_id`
- `planets` table has planet details

**Without JOINs:**
1. Query characters: Get `homeworld_id = 1`
2. Query planets: Find that ID 1 = "Tatooine"
3. Manually connect the information ❌

**With JOINs:**
- One query returns character name AND planet name together! ✅

### Real-World Analogy:

Think of JOINs like connecting puzzle pieces:
- Each table is a separate puzzle
- Foreign keys are the connector tabs
- JOIN puts the puzzles together to see the full picture


## 🔗 Types of JOINs

| Join Type | Returns | Use When |
|-----------|---------|----------|
| **INNER JOIN** | Only matching rows from both tables | You want records that exist in both tables |
| **LEFT JOIN** | All rows from left table, matching from right | You want all from table A, even if no match in table B |
| **RIGHT JOIN** | All rows from right table, matching from left | SQLite doesn't support; use LEFT JOIN instead |

### Visual Representation:

```
INNER JOIN:
Table A:  [1, 2, 3]
Table B:  [2, 3, 4]
Result:   [2, 3]      ← Only matches

LEFT JOIN:
Table A:  [1, 2, 3]
Table B:  [2, 3, 4]
Result:   [1, 2, 3]   ← All from A, with B data where available (4 excluded)
```


## 🛠️ Setup: Connect to Database

**Run the next 2 cells:**

In [ ]:
# Load SQL magic extension
%load_ext sql

# Fix prettytable compatibility issue
import prettytable
try:
    # Try to access DEFAULT to see if it exists
    _ = prettytable.DEFAULT
except AttributeError:
    # If it doesn't exist, add it using SINGLE_BORDER
    from prettytable import SINGLE_BORDER
    prettytable.DEFAULT = SINGLE_BORDER

# Configure SQL magic settings
%config SqlMagic.autopandas = False
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

In [ ]:
%sql sqlite:///starwars.db

In [ ]:
%%sql
-- Reference: View all tables
SELECT 'CHARACTERS' as table_name; SELECT * FROM characters LIMIT 3;
SELECT 'PLANETS' as table_name; SELECT * FROM planets LIMIT 3;


## 📚 Tables for JOIN Practice

**characters table:**
- Luke Skywalker → homeworld = 1 (Tatooine)
- Darth Vader → homeworld = 1 (Tatooine)
- Leia Organa → homeworld = 2 (Alderaan)
- R2-D2 → homeworld = 4 (Naboo)

**planets table:**
- id=1: Tatooine (desert)
- id=2: Alderaan (temperate)
- id=3: Kashyyyk (tropical)
- id=4: Naboo (temperate)

**vehicles table:**
- X-wing, Millennium Falcon, Speeder bike, TIE Fighter

**character_vehicles junction:**
- Links characters to vehicles they pilot

💡 **JOINs connect these tables using id = homeworld relationships!**


## 🤝 Part 1: INNER JOIN

### INNER JOIN Syntax

```sql
SELECT columns
FROM table1
INNER JOIN table2 ON table1.column = table2.column;
```

**Key Parts:**
- `FROM table1` - Start with first table (left table)
- `INNER JOIN table2` - Connect to second table (right table)
- `ON table1.column = table2.column` - Specify how they relate

### Query 1: Join Characters and Planets

**Goal:** Show characters with their homeworld details.


In [ ]:
%%sql
-- View data without JOIN (data from two tables)


**Explanation:**

- `characters.name` - Specify which table the column comes from (both tables have "name")
- `AS character_name` - Rename column in results (avoids confusion)
- `ON characters.homeworld_id = planets.id` - Links character's foreign key to planet's primary key

**Result:** Each row shows a character with their planet's information!


### Query 2: Using Table Aliases

Table names can be long. Use **aliases** to shorten them:

**Benefits:**
- Less typing
- Cleaner code
- Easier to read


In [ ]:
%%sql
-- INNER JOIN: Characters with their homeworld names


**Much cleaner!** 

- `FROM characters c` - `c` is shorthand for `characters`
- `INNER JOIN planets p` - `p` is shorthand for `planets`
- Now use `c.` and `p.` prefixes instead of full table names


### Query 3: Join Three Tables

Let's combine characters, vehicles, and the junction table:

**Path:** characters → character_vehicles → vehicles


In [ ]:
%%sql
-- Explicit INNER JOIN syntax


**Explanation:**

1. Start with `characters` table (alias `c`)
2. Join to `character_vehicles` junction table (alias `cv`) - links characters to vehicles
3. Join to `vehicles` table (alias `v`) - gets vehicle details
4. Order by character name

**Result:** Shows each character-vehicle pairing.


### Query 4: JOIN with WHERE

Combine JOINs with filtering:

**Order of Operations:**
1. JOIN tables first
2. Then filter with WHERE


In [ ]:
%%sql
-- INNER JOIN with table aliases


### Query 5: JOIN with Aggregate Functions

Count how many characters are from each planet:


In [ ]:
%%sql
-- Filter joined data: Humans with their homeworlds


**Important Note:** This only shows planets that HAVE characters (INNER JOIN requirement).

Planets with zero characters won't appear. We'll fix this with LEFT JOIN next!


## ⬅️ Part 2: LEFT JOIN

LEFT JOIN returns **ALL rows from the left table**, plus matching rows from the right table. If there's no match, NULL appears for right table columns.

### LEFT JOIN Syntax

```sql
SELECT columns
FROM table1
LEFT JOIN table2 ON table1.column = table2.column;
```

### When to Use LEFT JOIN:

- "Show all customers, even those with no orders"
- "List all students, even those not enrolled in courses"
- "Display all characters, even those without vehicles"

### Query 6: Characters and Their Vehicles (Including Those with None)


In [ ]:
%%sql
-- LEFT JOIN: All characters, with homeworld names where available


**Result:** Characters without vehicles show `NULL` for vehicle_name.

**Compare with INNER JOIN:** INNER JOIN would exclude characters with no vehicles entirely.


### Query 7: Find Characters WITHOUT Vehicles

**Technique:** After LEFT JOIN, use `WHERE column IS NULL` to find unmatched rows.


In [ ]:
%%sql
-- Find characters with no homeworld data


**Explanation:**

1. LEFT JOIN ensures all characters appear
2. Characters without vehicle links have `NULL` in `cv.vehicle_id`
3. WHERE filters to only those NULL rows

This is a common pattern for finding "missing" relationships!


### Query 8: Find Vehicles Without Pilots

**Flip the perspective:** Start with vehicles as the left table.


In [ ]:
%%sql
-- RIGHT JOIN: All planets, with characters where available


### Query 9: Count Including Empty Groups

**Compare with Query 5:** This time, include planets with 0 characters.


In [ ]:
%%sql
-- Find planets with no characters


**Key Difference from Query 5:**

- **Query 5 (INNER JOIN):** Only planets with characters
- **Query 9 (LEFT JOIN):** ALL planets, with count = 0 for empty ones

**Note:** `COUNT(c.id)` counts non-NULL values. Planets without characters have `c.id = NULL`, so count = 0.


## 🔗 Part 3: Complex JOIN Queries

### Query 10: Multiple JOINs with Multiple Filters

Find humans who pilot starfighters:


In [ ]:
%%sql
-- Join characters and vehicles through junction table


### Query 11: JOIN with HAVING

Find characters who pilot more than one vehicle:


In [ ]:
%%sql
-- LEFT JOIN to see all vehicles


**Explanation:**

- JOIN tables
- GROUP BY character
- COUNT vehicles per character
- HAVING filters to only those with 2+ vehicles


### Query 12: Comprehensive Character Summary

Combine all information about each character:


In [ ]:
%%sql
-- Three-way JOIN: Characters, homeworld, and vehicle


**Why LEFT JOIN here?**

We want ALL characters, even if they:
- Don't have a homeworld recorded
- Don't pilot any vehicles

Using INNER JOIN would exclude them.


## 📊 INNER JOIN vs LEFT JOIN Comparison

### Example: Count Vehicles per Manufacturer

**INNER JOIN:**


In [ ]:
%%sql
-- Exercise 1: Find all droids and their homeworlds


**LEFT JOIN:**


In [ ]:
%%sql
-- Exercise 2: Count characters by planet


**Difference:**
- INNER JOIN excludes manufacturers whose vehicles aren't piloted
- LEFT JOIN includes them with count = 0


## 🎓 Practice Exercises

### Exercise 1: Simple INNER JOIN

List all characters with their homeworld's population:


In [ ]:
%%sql
-- Exercise 3: Find characters from desert planets


### Exercise 2: Multiple JOINs

Show all vehicle-pilot pairs with character species:


In [ ]:
%%sql
-- Exercise 4: Find tallest character on each planet


### Exercise 3: LEFT JOIN with NULL Check

Find all planets with no characters:


In [ ]:
%%sql
-- Exercise 5: List vehicles and who flies them


### Exercise 4: Aggregate with JOIN

Show each vehicle with the count of who pilots it:


In [ ]:
%%sql
-- Challenge 1: Characters with vehicles from desert planets


## 🐛 Common Errors & Troubleshooting

### Error: "ambiguous column name"

**Problem:** Column exists in multiple tables and you didn't specify which.

**Wrong:**
```sql
SELECT name FROM characters
INNER JOIN planets ON homeworld_id = id;
```

**Correct:**
```sql
SELECT characters.name FROM characters
INNER JOIN planets ON characters.homeworld_id = planets.id;
```

**Best Practice:** Always use table prefixes (or aliases) in JOINs.


### Error: "no such column"

**Problem:** Misspelt column or using wrong table prefix.

**Solution:** Verify column names:


In [ ]:
%%sql
-- Challenge 2: Self-join example setup


In [ ]:
%%sql
-- Challenge 2: Self-join query


### Wrong JOIN Type

**Symptom:** Missing expected rows in results.

**Problem:** Used INNER JOIN when you needed LEFT JOIN.

**Remember:**
- **INNER JOIN:** Only rows with matches in BOTH tables
- **LEFT JOIN:** ALL rows from left table, matches from right

**Example:**
```sql
-- ❌ Missing characters without vehicles
SELECT c.name, v.name
FROM characters c
INNER JOIN character_vehicles cv ON c.id = cv.character_id
INNER JOIN vehicles v ON cv.vehicle_id = v.id;

-- ✅ Shows all characters, even without vehicles
SELECT c.name, v.name
FROM characters c
LEFT JOIN character_vehicles cv ON c.id = cv.character_id
LEFT JOIN vehicles v ON cv.vehicle_id = v.id;
```


### Incorrect ON Clause

**Problem:** Joining on wrong columns.

**Wrong:**
```sql
FROM characters c
INNER JOIN planets p ON c.id = p.id  -- ❌ Wrong columns!
```

**Correct:**
```sql
FROM characters c
INNER JOIN planets p ON c.homeworld_id = p.id  -- ✅ Foreign key to primary key
```

**Always JOIN:**
- Foreign key → Primary key
- Example: `table1.foreign_key_id = table2.id`


### Cartesian Product (Too Many Results)

**Problem:** Missing ON clause creates every possible combination.

**Wrong:**
```sql
-- ❌ This creates 11 characters × 8 planets = 88 rows!
SELECT * FROM characters, planets;
```

**Correct:**
```sql
-- ✅ Only creates valid character-planet pairs
SELECT * FROM characters
INNER JOIN planets ON characters.homeworld_id = planets.id;
```

**Rule:** Always include the ON clause with JOIN!


## 🎯 Challenge Problem

**Task:** Create a query that shows each planet with:
- Planet name
- Climate
- Number of characters from that planet
- Average height of characters from that planet
- Include planets with NO characters (show 0 for count, NULL for average)
- Order by character count (descending), then planet name

**Requirements:**
- Use LEFT JOIN
- Use COUNT() and AVG()
- Use GROUP BY
- Use ORDER BY with multiple columns
- Use ROUND() for avg_height (1 decimal place)


In [ ]:
%%sql
-- Challenge 3: Complex multi-way join


## ✅ Checkpoint & Summary

### What You've Learnt

- ✅ Understand the purpose of JOINs (combining related tables)
- ✅ Write INNER JOIN queries (only matching rows)
- ✅ Write LEFT JOIN queries (all from left, matches from right)
- ✅ Use table aliases for cleaner code (`FROM characters c`)
- ✅ Join three or more tables in sequence
- ✅ Combine JOINs with WHERE, GROUP BY, and HAVING
- ✅ Find unmatched records using LEFT JOIN + `IS NULL`
- ✅ Count aggregates including empty groups

### Key SQL Commands

| Command | Purpose | Example |
|---------|---------|---------|
| `INNER JOIN` | Return only matching rows | `INNER JOIN planets ON c.homeworld_id = p.id` |
| `LEFT JOIN` | Return all from left table | `LEFT JOIN vehicles ON c.id = v.pilot_id` |
| `ON` | Specify join condition | `ON table1.foreign_key_id = table2.id` |
| Table Alias | Shorten table names | `FROM characters c` |
| `IS NULL` | Check for NULL values | `WHERE cv.vehicle_id IS NULL` |
| Multiple JOINs | Chain joins | `JOIN t1 ... JOIN t2 ... JOIN t3` |

### JOIN Decision Tree

**Question:** Do I need ALL rows from one table?
- **Yes** → Use LEFT JOIN (with that table on left)
- **No** → Use INNER JOIN

**Question:** Which table should be on the left?
- Put the table you want "all rows from" on the left
- Use LEFT JOIN

## 🎉 Excellent Work!

You can now combine data from multiple tables and retrieve complex information! In the next lesson, you'll learn how to modify data with UPDATE and DELETE statements, and work with transactions.

**Ready to continue?** Open `lesson7_modifications.ipynb`

---

## 💾 Git Commands (for reference)

```bash
git status
git add solutions/lesson6_joins.ipynb
git commit -m "Completed Lesson 6: INNER and LEFT joins"
git push
```
